# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770932283655                   -0.52    8.0         
  2   -2.772145630722       -2.92       -1.32    1.0    153ms
  3   -2.772170208495       -4.61       -2.49    1.0    131ms
  4   -2.772170656155       -6.35       -3.18    2.0    151ms
  5   -2.772170722195       -7.18       -4.09    1.0    151ms
  6   -2.772170722953       -9.12       -4.87    1.0    137ms
  7   -2.772170723014      -10.21       -5.60    2.0    177ms
  8   -2.772170723015      -12.22       -6.14    2.0    161ms
  9   -2.772170723015      -13.70       -6.76    1.0    163ms
 10   -2.772170723015   +  -13.91       -6.95    1.0    150ms
 11   -2.772170723015      -14.10       -7.64    1.0    145ms
 12   -2.772170723015   +  -14.12       -8.36    2.0    182ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.7

1.7735580511599087

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770489838607                   -0.53    9.0         
  2   -2.772054667979       -2.81       -1.29    1.0    196ms
  3   -2.772082772574       -4.55       -2.70    1.0    132ms
  4   -2.772083416199       -6.19       -3.99    2.0    166ms
  5   -2.772083417767       -8.80       -4.83    1.0    169ms
  6   -2.772083417810      -10.37       -5.49    2.0    156ms
  7   -2.772083417811      -11.98       -6.15    1.0    139ms
  8   -2.772083417811      -13.34       -7.15    1.0    169ms
  9   -2.772083417811      -14.21       -7.44    2.0    167ms
 10   -2.772083417811   +  -13.81       -8.89    1.0    167ms

Polarizability via ForwardDiff:       1.772534972943695
Polarizability via finite difference: 1.7735580511599087
